# Bibliotecas

In [3]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Leitura dos Dados

In [4]:
endereco = '../BaseDadosSelecionados/'
# endereco2 = 'Agrupado_Ocorrencia'
nomeArq = 'ES_datatran2019'

In [5]:
data = pd.read_csv(endereco + nomeArq + '.csv', encoding='ISO-8859-1', sep=';')

In [6]:
data.shape

(2842, 26)

In [7]:
data.head()

,id,data_inversa,dia_semana,horario,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,...,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude
0,182269.0,2019-01-01,terça-feira,10:00:00,101.0,"121,3",SOORETAMA,Condutor Dormindo,Colisão lateral,Com Vítimas Feridas,...,2,0,0,1,1,0,1,2,"-19,15490874","-40,08092881"
1,182296.0,2019-01-01,terça-feira,13:40:00,262.0,95,DOMINGOS MARTINS,Não guardar distância de segurança,Engavetamento,Sem Vítimas,...,4,0,0,0,4,0,0,4,"-20,3677","-41,0595"
2,182319.0,2019-01-01,terça-feira,16:00:00,101.0,"77,1",SAO MATEUS,Falta de Atenção à Condução,Colisão traseira,Com Vítimas Feridas,...,3,0,1,1,1,0,2,2,"-18,720545","-39,865883"
3,182350.0,2019-01-01,terça-feira,18:30:00,101.0,"299,2",VIANA,Desobediência às normas de trânsito pelo condutor,Colisão transversal,Sem Vítimas,...,3,0,0,0,3,0,0,2,"-20,35788487","-40,42651176"
4,182351.0,2019-01-01,terça-feira,18:10:00,262.0,168,IBATIBA,Falta de Atenção à Condução,Queda de ocupante de veículo,Com Vítimas Feridas,...,1,0,0,1,0,0,1,1,"-20,265","-41,5732"


# Carregamento dos dados em uma matriz

In [8]:
# data.shape[0] LINHAS e data.shape[1] COLUNAS
dataset = []
for i in range(0, data.shape[0]):
    dataset.append([str(data.values[i,j]) for j in range(0, data.shape[1])])

# Execução do algoritmo Apriori

In [9]:
%%time
apriori(sparse_df, min_support=numMinSupport, use_colnames=True, verbose=1)

NameError: name 'sparse_df' is not defined

In [10]:
%%time
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = apriori(df, min_support=numMinSupport, use_colnames=True)

# frequent_itemsets

NameError: name 'numMinSupport' is not defined

# Salvando os resultados em um .csv

In [11]:
association_rules_saida = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

NameError: name 'frequent_itemsets' is not defined

In [ ]:
saidaRecords = pd.DataFrame(association_rules_saida)

In [ ]:
saidaRecords.head(10)

In [ ]:
association_rules_saida.to_csv('AprioriMlxtend_'+ nomeArq + '.csv', encoding= 'ISO-8859-1', sep= ';', index=False)

In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
rules

## Método para contabilizaro número de consequentes e antecedentes em cada regra

In [ ]:
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules

In [ ]:
rules["consequents_len"] = rules["consequents"].apply(lambda x: len(x))
rules

### Filtro para facilitar a visualização dos resultados

In [ ]:
rules[ (rules['antecedent_len'] >= 2) &
       (rules['confidence'] > 0.75) &
       (rules['lift'] > 1.2) ]

In [ ]:
# rules[rules['antecedents'] == {'Eggs', 'Kidney Beans'}]

# Nova execução do algoritmo apriori - Apresentação diferente

In [ ]:
apriori(df, min_support=numMinSupport)

In [ ]:
%%time
apriori(df, min_support=numMinSupport, use_colnames=True)

In [ ]:
frequent_itemsets = apriori(df, min_support=numMinSupport, use_colnames=True, verbose=1)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

## Filtro referente ao último método apresentado

In [ ]:
frequent_itemsets[ (frequent_itemsets['length'] == 2) &
                   (frequent_itemsets['support'] >= 0.8) ]

In [ ]:
# frequent_itemsets[ frequent_itemsets['itemsets'] == {'Onion', 'Eggs'} ]